In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
"""impot data select features"""
data = pd.read_csv("application_training_data_import.csv")
features = data.columns
s=data.isna().sum()
null = pd.read_csv('Null_Entries.csv')
d = null[null['flag']==0]
columns = d['colname']
new_data = data[columns]



home_loans = new_data[new_data.isnull().any(axis=1) == False]
home_loans = home_loans.reset_index()
home_loans = home_loans.drop(columns = 'index')

#print(home_loans.shape)


cols_to_remove = ['SK_ID_CURR',
'FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5',
'FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8',
'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12',
'FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
'FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21',
'AMT_REQ_CREDIT_BUREAU_HOUR',
'AMT_REQ_CREDIT_BUREAU_DAY',
'AMT_REQ_CREDIT_BUREAU_WEEK',
'FLAG_CONT_MOBILE']


cols_to_normalize = ['EXT_SOURCE_3',
'EXT_SOURCE_2','AMT_GOODS_PRICE',
'AMT_ANNUITY','DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL',
'AMT_CREDIT','REGION_POPULATION_RELATIVE',
'DAYS_BIRTH',
'DAYS_EMPLOYED',
'DAYS_REGISTRATION',
'DAYS_ID_PUBLISH']


"""drop columns and add normalized columns"""
home_loans_df = home_loans.drop(cols_to_remove, axis=1)
## substitute the normalized columns
df_to_normalize = home_loans_df[cols_to_normalize]
norm_values = df_to_normalize.values

"""normalize values"""
scaler = StandardScaler()
scaler.fit(norm_values)
norm_values= scaler.transform(norm_values)
norm_values

"""re-impute into data"""
home_loans_df[cols_to_normalize] = norm_values
home_loans_df.head()

,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_2,AMT_GOODS_PRICE,...,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY
0,-1.905793,0.0,0.0,1.0,2.0,2.0,2.0,2.0,-1.351241,-0.527764,...,2,2,WEDNESDAY,10,0,0,0,0,0,0
1,1.123108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.197991,-1.109049,...,2,2,MONDAY,9,0,0,0,0,0,0
2,0.567098,0.0,1.0,1.0,0.0,0.0,0.0,0.0,-0.868560,-0.249232,...,2,2,WEDNESDAY,16,0,0,0,0,0,0
3,-0.095792,1.0,1.0,2.0,1.0,0.0,1.0,0.0,1.086863,2.281779,...,2,2,SUNDAY,16,0,0,0,0,0,0
4,0.153597,0.0,0.0,0.0,2.0,0.0,2.0,0.0,1.035459,2.645082,...,3,3,MONDAY,16,0,0,0,0,1,1


In [3]:
"""Set the indices"""
n0 = home_loans_df[home_loans_df['TARGET']==0].shape[0]
#test set percentage size 
test_per = 0.0758
n1 = home_loans_df[home_loans_df['TARGET']==1].shape[0]
extra = 3000

train1_index = 14425
train0_index = int(train1_index)

test1_index = int(n1 - train1_index)
test0_index = int(test1_index/test_per)
#test0_index = test0_index+test1_index
# print(test1_index)
# print(test0_index)

"""Take all TARGET Home Loans"""
home_loans = shuffle(home_loans_df,random_state=0)
home_loans0 = home_loans[home_loans['TARGET']==0].iloc[:108200, :]
home_loans1 = home_loans[home_loans['TARGET']==1].iloc[:, :]

# home_loans_test0 = home_loans[home_loans['TARGET']==0].iloc[10000:101000, :]
# home_loans_test1 = home_loans[home_loans['TARGET']==1].iloc[:, :]

reduced_loans = pd.concat([home_loans0, home_loans1], axis=0)
reduced_loans = shuffle(reduced_loans,random_state=0)


reduced_loans = pd.get_dummies(reduced_loans)


##########
################
########---Split data into Test-Train Split
####################
########################
"""split dataset into training set and cross validation"""
"""training data will have 50/50 0/1 class split"""
train_df0 = reduced_loans[reduced_loans['TARGET']==0].iloc[:train0_index, :]
train_df1 = reduced_loans[reduced_loans['TARGET']==1].iloc[:train1_index, :]

train = pd.concat([train_df0, train_df1])
train = shuffle(train,random_state=43)


"""Test dataframe will have 90/10 0/1 class split"""
test_df0 = reduced_loans[reduced_loans['TARGET']==0].iloc[test0_index:, :]
test_df1 = reduced_loans[reduced_loans['TARGET']==1].iloc[test1_index:, :]

test = pd.concat([test_df0, test_df1])
test = shuffle(test,random_state=57)

del reduced_loans, home_loans, home_loans0, home_loans1
######
#########
#######-----split up the y variables
##########


"""separate out the target variable"""
train_target = train['TARGET']
train = train.drop(columns = 'TARGET')
train = pd.get_dummies(train) #One Hot Encoding
# train = train.drop(columns='NAME_INCOME_TYPE_Unemployed')
# train = train.drop(columns='GENDER_CODE_XNA')

test_target = test['TARGET']
test = test.drop(columns = 'TARGET')
test = pd.get_dummies(test) #One Hot Encoding



"""break up the data"""
X_train, y_train = np.matrix(train), np.array(train_target)
X_test, y_test = np.matrix(test), np.array(test_target)

#######
#######---Create cross validation sets-------######
########
#########

traincv1 = X_train[:int(np.floor((len(train_target)/3))), :]
target1 = y_train[:int(np.floor((len(train_target)/3)))]

traincv2 = X_train[int(np.floor((len(train_target)/3))):int(np.floor((len(train_target)*2/3))), :]
target2 = y_train[int(np.floor((len(train_target)/3))):int(np.floor((len(train_target)*2/3)))]


traincv3 = X_train[int(np.floor((len(train_target)*2/3))):, :]
target3 = y_train[int(np.floor((len(train_target)*2/3))):]


"""add test dataset"""
# test = X[int(np.floor((len(reduced_loans)*3/4))):, :]
# target_test = y[int(np.floor((len(reduced_loans)*3/4))):]


"""Trainig """
print('Training data shaped {}'.format(X_train.shape))
print(traincv1.shape)

print('Testing data shaped {}'.format(X_test.shape))


"""make pairs of datasets"""
CV12 = np.concatenate([traincv1, traincv2])
y12 = np.concatenate([target1, target2])

CV13 = np.concatenate([traincv1, traincv3])
y13 = np.concatenate([target1, target3])

CV23 = np.concatenate([traincv2, traincv3])
y23 = np.concatenate([target2, target3])

print('cross validation shapes')
print(CV12.shape)

Training data shaped (28850, 67)
(9616, 67)
Testing data shaped (61333, 67)
cross validation shapes
(19233, 67)


### Test on 1, 2

In [4]:
support_vector_machine12 = SVC(kernel='rbf', gamma=0.1, C=1., random_state=None)
support_vector_machine12.fit(CV12, y12)
print('finished training on 1,2')

#y_scores = clf.decision_function(X_test)
y_scores_train = support_vector_machine12.predict(CV12)
y_scores_test = support_vector_machine12.predict(traincv3)

confusion_train = confusion_matrix(y12, y_scores_train)
confusion_test = confusion_matrix(target3, y_scores_test)


print('Trained on 1,2')
print('Train Confusion Matrix with accuracy {}'.format((confusion_train[0][0]+confusion_train[1][1])/len(y12)))
print('Train Recall: {}'.format((confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]))))
print(confusion_train)
print('')
print('Test Confusion Matrix with accuracy {}'.format((confusion_test[0][0]+confusion_test[1][1])/len(target3)))
print('Test Recall: {}'.format((confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]))))
print(confusion_test)
#precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

finished training on 1,2
Trained on 1,2
Train Confusion Matrix with accuracy 0.9103623979618364
Train Recall: 0.9065371762740184
[[8828  829]
 [ 895 8681]]

Test Confusion Matrix with accuracy 0.6705833419985442
Test Recall: 0.657042689214271
[[3263 1505]
 [1663 3186]]


### Test 1, 3

In [5]:
support_vector_machine13 = SVC(kernel='rbf', gamma=0.1, C=1., random_state=None)
support_vector_machine13.fit(CV13, y13)
print('finished training on 1,3')

#y_scores = clf.decision_function(X_test)
y_scores_train = support_vector_machine13.predict(CV13) 
y_scores_test = support_vector_machine13.predict(traincv2)

confusion_train = confusion_matrix(y13, y_scores_train)
confusion_test = confusion_matrix(target2, y_scores_test)


print('Trained on 1,3')
print('Train Confusion Matrix with accuracy {}'.format((confusion_train[0][0]+confusion_train[1][1])/len(y13)))
print('Train Recall: {}'.format((confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]))))
print(confusion_train)
print('')
print('Test Confusion Matrix with accuracy {}'.format((confusion_test[0][0]+confusion_test[1][1])/len(target2)))
print('Test Recall: {}'.format((confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]))))
print(confusion_test)
#precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

finished training on 1,3
Trained on 1,3
Train Confusion Matrix with accuracy 0.9084906150886497
Train Recall: 0.9041853668719341
[[8810  842]
 [ 918 8663]]

Test Confusion Matrix with accuracy 0.6716231673078923
Test Recall: 0.6717588769611891
[[3205 1568]
 [1590 3254]]


### Test 2,3

In [6]:
support_vector_machine23 = SVC(kernel='rbf', gamma=0.1, C=1., random_state=None)
support_vector_machine23.fit(CV23, y23)
print('finished training on 2,3')
#y_scores = clf.decision_function(X_test)
y_scores_train = support_vector_machine23.predict(CV23)
y_scores_test = support_vector_machine23.predict(traincv1)

confusion_train = confusion_matrix(y23, y_scores_train)
confusion_test = confusion_matrix(target1, y_scores_test)


print('Trained on 2,3')
print('Train Confusion Matrix with accuracy {}'.format((confusion_train[0][0]+confusion_train[1][1])/len(y23)))
print('Train Recall: {}'.format((confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]))))
print(confusion_train)
print('')
print('Test Confusion Matrix with accuracy {}'.format((confusion_test[0][0]+confusion_test[1][1])/len(target1)))
print('Test Recall: {}'.format((confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]))))
print(confusion_test)
#precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

finished training on 2,3
Trained on 2,3
Train Confusion Matrix with accuracy 0.9118228137672871
Train Recall: 0.9072526565562777
[[8744  797]
 [ 899 8794]]

Test Confusion Matrix with accuracy 0.6644134775374376
Test Recall: 0.6464497041420119
[[3330 1554]
 [1673 3059]]


### Train Best Model and Test on Testing Data

In [7]:
"""After Eyeballing the model with the best Recall"""
support_vector_machine_best = support_vector_machine13
support_vector_machine_best.fit(X_train, y_train)
print('finished training on FULL Training Data')
#y_scores = clf.decision_function(X_test)
y_scores_train = support_vector_machine_best.predict(X_train)
y_scores_test = support_vector_machine_best.predict(X_test)

confusion_train = confusion_matrix(y_train, y_scores_train)
confusion_test = confusion_matrix(y_test, y_scores_test)


print('Trained on FULL Training Data')
print('Train Confusion Matrix with accuracy {}'.format((confusion_train[0][0]+confusion_train[1][1])/len(y_train)))
print('Train Recall: {}'.format((confusion_train[1][1]/(confusion_train[1][0]+confusion_train[1][1]))))
print(confusion_train)
print('')
print('Test Confusion Matrix with accuracy {}'.format((confusion_test[0][0]+confusion_test[1][1])/len(y_test)))
print('Test Recall: {}'.format((confusion_test[1][1]/(confusion_test[1][0]+confusion_test[1][1]))))
print(confusion_test)
#precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

finished training on FULL Training Data
Trained on FULL Training Data
Train Confusion Matrix with accuracy 0.8984055459272097
Train Recall: 0.8944887348353553
[[13016  1409]
 [ 1522 12903]]

Test Confusion Matrix with accuracy 0.7104495133125723
Test Recall: 0.8205892547660312
[[31737 15171]
 [ 2588 11837]]
